# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 10:47AM,247146,16,TASA-280061,TASA USA Inc.,Released
1,Sep 23 2022 10:39AM,247145,19,MSP214251,"Methapharm, Inc.",Released
2,Sep 23 2022 10:36AM,247144,10,SONSB0001952,"Nextsource Biotechnology, LLC",Released
3,Sep 23 2022 10:36AM,247144,10,SOTAH0000787,"Nextsource Biotechnology, LLC",Released
4,Sep 23 2022 10:32AM,247143,20,ALU0007987,Alumier Labs Inc.,Released
5,Sep 23 2022 10:32AM,247143,20,ALU0007994,Alumier Labs Inc.,Released
6,Sep 23 2022 10:32AM,247143,20,ALU0007995,Alumier Labs Inc.,Released
7,Sep 23 2022 10:32AM,247143,20,ALUR176077994,Alumier Labs Inc.,Released
8,Sep 23 2022 10:32AM,247143,20,ALUR474981228,Alumier Labs Inc.,Released
9,Sep 23 2022 10:32AM,247143,20,ALUR833234443,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,247142,Released,1
31,247143,Released,6
32,247144,Released,2
33,247145,Released,1
34,247146,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247142,NaN,NaN,1.0
247143,NaN,NaN,6.0
247144,NaN,NaN,2.0
247145,NaN,NaN,1.0
247146,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
246999,0.0,1.0,0.0
247000,0.0,1.0,0.0
247048,2.0,1.0,32.0
247059,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
246999,0,1,0
247000,0,1,0
247048,2,1,32
247059,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,246999,0,1,0
2,247000,0,1,0
3,247048,2,1,32
4,247059,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,246999,,1,
2,247000,,1,
3,247048,2,1,32
4,247059,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc."
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC"
4,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.
10,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC"
11,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc."
12,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc."
13,Sep 23 2022 10:10AM,247140,10,Bio-PRF
19,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.
28,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,,,1
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc.",,,1
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC",,,2
3,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,,,6
4,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",,,1
5,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",,,1
6,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",,1,
7,Sep 23 2022 10:10AM,247140,10,Bio-PRF,,1,5
8,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.,,,9
9,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,1,,
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc.",1,,
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC",2,,
3,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6,,
4,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1,,
5,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",1,,
6,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",,1,
7,Sep 23 2022 10:10AM,247140,10,Bio-PRF,5,1,
8,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.,9,,
9,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,1,,
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc.",1,,
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC",2,,
3,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6,,
4,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,1.0,NaN,NaN
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc.",1.0,NaN,NaN
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
3,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6.0,NaN,NaN
4,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,1.0,0.0,0.0
1,Sep 23 2022 10:39AM,247145,19,"Methapharm, Inc.",1.0,0.0,0.0
2,Sep 23 2022 10:36AM,247144,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
3,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6.0,0.0,0.0
4,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2471279,92.0,4.0,5.0
16,741394,3.0,0.0,0.0
19,1235613,7.0,0.0,0.0
20,741311,45.0,30.0,11.0
21,1482180,2.0,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2471279,92.0,4.0,5.0
1,16,741394,3.0,0.0,0.0
2,19,1235613,7.0,0.0,0.0
3,20,741311,45.0,30.0,11.0
4,21,1482180,2.0,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,92.0,4.0,5.0
1,16,3.0,0.0,0.0
2,19,7.0,0.0,0.0
3,20,45.0,30.0,11.0
4,21,2.0,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,16,Released,3.0
2,19,Released,7.0
3,20,Released,45.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,5.0,0.0,0.0,11.0,0.0
Executing,4.0,0.0,0.0,30.0,4.0
Released,92.0,3.0,7.0,45.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,11.0,0.0
1,Executing,4.0,0.0,0.0,30.0,4.0
2,Released,92.0,3.0,7.0,45.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,11.0,0.0
1,Executing,4.0,0.0,0.0,30.0,4.0
2,Released,92.0,3.0,7.0,45.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()